In [2]:
import pandas as pd

In [3]:
dff_meta = pd.read_csv('../Dataset/trafficMetaData.csv')
dff_meta.head(5)

,POINT_1_STREET,DURATION_IN_SEC,POINT_1_NAME,POINT_1_CITY,POINT_2_NAME,POINT_2_LNG,POINT_2_STREET,NDT_IN_KMH,POINT_2_POSTAL_CODE,POINT_2_COUNTRY,...,POINT_2_CITY,extID,ROAD_TYPE,POINT_1_LNG,REPORT_ID,POINT_1_COUNTRY,DISTANCE_IN_METERS,REPORT_NAME,RBA_ID,_id
0,Søftenvej,77,4364,Aarhus,4349,10.161038,Søftenvej,70,8200,Denmark,...,Aarhus,179,MAJOR_ROAD,10.139696,158895,Denmark,1505,AAR_BT_0020 4364 4349,161062,1586
1,Grenåvej,65,4582,Risskov,4581,10.219086,Grenåvej,44,8240,Denmark,...,Risskov,194,MAJOR_ROAD,10.218304,178600,Denmark,797,AAR_BT_0101 4582 4581,178660,1601
2,Skanderborgvej,84,3987,Aarhus,2651,10.157659,Viby Ringvej,35,8260,Denmark,...,Aarhus,324,MAJOR_ROAD,10.166163,187509,Denmark,823,AAR_BT_0231 3987 2651,189702,1731
3,Grenåvej,55,4572,Risskov,4581,10.218878,Grenåvej,53,8240,Denmark,...,Risskov,383,MAJOR_ROAD,10.228594,190744,Denmark,813,AAR_BT_0291 4572 4581,192392,1790
4,Holmstrupgårdvej,91,4337,Aarhus,4336,10.120842,Holmstrupgårdvej,40,8210,Denmark,...,Aarhus,436,MAJOR_ROAD,10.119669,195041,Denmark,1015,AAR_BT_0356 4337 4336,229156,1843


In [4]:
df_detail = pd.read_csv('../Dataset/traffic_feb_june/trafficData158324.csv')
df_detail.head(5)

,status,avgMeasuredTime,avgSpeed,extID,medianMeasuredTime,TIMESTAMP,vehicleCount,_id,REPORT_ID
0,OK,66,56,668,66,2014-02-13T11:30:00,7,190000,158324
1,OK,69,53,668,69,2014-02-13T11:35:00,5,190449,158324
2,OK,69,53,668,69,2014-02-13T11:40:00,6,190898,158324
3,OK,70,52,668,70,2014-02-13T11:45:00,3,191347,158324
4,OK,64,57,668,64,2014-02-13T11:50:00,6,191796,158324


In [5]:
df_detail['date'] = df_detail['TIMESTAMP'].apply(lambda x: x.split('T')[0])
df_detail['time'] = df_detail['TIMESTAMP'].apply(lambda x: x.split('T')[1])
df_detail.drop('TIMESTAMP', axis=1, inplace=True)
df_detail.head(5)

,status,avgMeasuredTime,avgSpeed,extID,medianMeasuredTime,vehicleCount,_id,REPORT_ID,date,time
0,OK,66,56,668,66,7,190000,158324,2014-02-13,11:30:00
1,OK,69,53,668,69,5,190449,158324,2014-02-13,11:35:00
2,OK,69,53,668,69,6,190898,158324,2014-02-13,11:40:00
3,OK,70,52,668,70,3,191347,158324,2014-02-13,11:45:00
4,OK,64,57,668,64,6,191796,158324,2014-02-13,11:50:00


In [6]:
df_detail['time'] = df_detail['time'].apply(lambda x: x.split(':')[0])

In [7]:
df_detail.drop(columns=['status','extID','medianMeasuredTime'], inplace= True)
df_group = df_detail.groupby(['date', 'time']).agg({'avgMeasuredTime': 'mean','vehicleCount': 'sum'}).reset_index()
df_group.head(10)

,date,time,avgMeasuredTime,vehicleCount
0,2014-02-13,11,68.833333,36
1,2014-02-13,12,62.500000,124
2,2014-02-13,13,62.500000,92
3,2014-02-13,14,66.333333,110
4,2014-02-13,15,64.833333,91
5,2014-02-13,16,63.250000,69
6,2014-02-13,17,64.583333,49
7,2014-02-13,18,58.500000,17
8,2014-02-13,19,68.250000,11
9,2014-02-13,20,58.500000,3


In [9]:
print(type(df_group['time'][0]))

<class 'str'>


In [84]:
import json

In [96]:
def get_weather_data(file, type):
    json_data = []
    with open(file) as f:
        for line in f:
            json_data.append(json.loads(line))
    pd_json = pd.DataFrame()
    flatten_data =[]
    for item in json_data:    
        df_temp = pd.DataFrame()
        df_temp['TIMESTAMP'] = item.keys()
        df_temp[type] = item.values()
        flatten_data.append(df_temp)
    pd_json = pd.concat(flatten_data, axis=0)
    return pd_json


In [97]:
#get all files in the folder
import os
import functools as ft
files = os.listdir('../Dataset/weather/')
dfs=[]
for file in files:
    dftmp = get_weather_data('../Dataset/weather/'+file,file.split('.')[0])
    dfs.append(dftmp)
df_final = ft.reduce(lambda left, right: pd.merge(left, right, on='TIMESTAMP'), dfs)

In [98]:
df_final.head(5)

,TIMESTAMP,dewptm,hum,pressurem,tempm,wdird,wspdm
0,2014-08-01T01:00:00,13,64,1012,18,210,7.4
1,2014-08-01T13:00:00,15,53,1013,22,180,11.1
2,2014-08-01T01:20:00,13.0,73,1012,18.0,210,7.4
3,2014-08-01T13:20:00,13.0,57,1013,22.0,180,14.8
4,2014-08-01T13:50:00,12.0,47,1013,24.0,170,14.8


In [100]:
df_final.describe()

,dewptm,hum,pressurem,tempm,wdird,wspdm,date,time
count,4357,4357,4357,4357,4357,4357,4357,4357
unique,41,76,40,51,38,23,61,72
top,11.0,82,1012,14.0,240,11.1,2014-08-01,01:00:00
freq,556,511,353,356,335,482,72,61


In [99]:
df_final['date'] = df_final['TIMESTAMP'].apply(lambda x: x.split('T')[0])
df_final['time'] = df_final['TIMESTAMP'].apply(lambda x: x.split('T')[1])
df_final.drop('TIMESTAMP', axis=1, inplace=True)
df_final.head(5)

,dewptm,hum,pressurem,tempm,wdird,wspdm,date,time
0,13,64,1012,18,210,7.4,2014-08-01,01:00:00
1,15,53,1013,22,180,11.1,2014-08-01,13:00:00
2,13.0,73,1012,18.0,210,7.4,2014-08-01,01:20:00
3,13.0,57,1013,22.0,180,14.8,2014-08-01,13:20:00
4,12.0,47,1013,24.0,170,14.8,2014-08-01,13:50:00
